# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [14]:
def calcular_valor(numero, palo):
    if numero == 1 and palo == "espada":
        return 14  
    elif numero == 1 and palo == "basto":
        return 13  
    elif numero == 7 and palo == "espada":
        return 12 
    elif numero == 7 and palo == "oro":
        return 11  
    elif numero == 3:
        return 10
    elif numero == 2:
        return 9 
    elif numero == 1 and palo in ["copa", "oro"]:
        return 8   
    elif numero == 12:
        return 7 
    elif numero == 11:
        return 6 
    elif numero == 10:
        return 5
    elif numero == 7 and palo in ["copa", "basto"]:
        return 4   
    elif numero == 6:
        return 3 
    elif numero == 5:
        return 2 
    elif numero == 4:
        return 1 
    else:
        return 0   

class Carta:
    def __init__(self, numero, palo):
        if numero not in range(1, 13) or numero in [8, 9]:
            raise ValueError("Número de carta inválido")
        if palo not in ["oro", "copa", "espada", "basto"]:
            raise ValueError("Palo inválido")
        
        self.numero = numero
        self.palo = palo
        self.valor = calcular_valor(numero, palo)
    
    def __str__(self):
        return f"{self.numero} de {self.palo}"
    
    def __repr__(self):
        return f"Carta({self.numero}, '{self.palo}')"

c = Carta(1, "oro")
print(c)
print(c.valor)


1 de oro
8


**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [15]:
import random

class Mazo:
    def __init__(self):
        self.cartas = []
        palos = ["oro", "copa", "espada", "basto"]
        numeros = [1, 2, 3, 4, 5, 6, 7, 10, 11, 12]
        for palo in palos:
            for numero in numeros:
                self.cartas.append(Carta(numero, palo))
    
    def mezclar(self):
        random.shuffle(self.cartas)
    
    def repartir(self, cantidad):
        if cantidad > len(self.cartas):
            raise ValueError("No hay suficientes cartas en el mazo")
        cartas_repartidas = self.cartas[:cantidad]
        self.cartas = self.cartas[cantidad:]
        return cartas_repartidas
    
    def __len__(self):
        return len(self.cartas)
    
    def __str__(self):
        return f"Mazo con {len(self)} cartas"


m = Mazo()
print(m) 
m.mezclar()
mano = m.repartir(3)
print([str(carta) for carta in mano])
print(m)

Mazo con 40 cartas
['11 de oro', '3 de oro', '7 de basto']
Mazo con 37 cartas


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [16]:
class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.mano = []
        self.puntos = 0
    
    def recibir_cartas(self, cartas):
        self.mano.extend(cartas)
    
    def jugar_carta(self):
        if not self.mano:
            raise ValueError("No hay cartas en la mano para jugar")
        carta_jugada = random.choice(self.mano)
        self.mano.remove(carta_jugada)
        return carta_jugada
    
    def sumar_punto(self):
        self.puntos += 1
    
    def __str__(self):
        return f"{self.nombre}: {self.puntos} Puntos"
    
    def __repr__(self):
        return f"Jugador('{self.nombre}')"


j = Jugador("Juan")
j.recibir_cartas(m.repartir(3))
print(j)
carta_jugada = j.jugar_carta()
print(f"{j.nombre} jugó: {carta_jugada}")
j.sumar_punto()
print(j)

Juan: 0 Puntos
Juan jugó: 6 de basto
Juan: 1 Puntos


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se terminen las cartas del mazo.

In [17]:
class Partida:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.mazo = Mazo()
        self.puntaje = {jugador1: 0, jugador2: 0}

    def jugar_ronda(self):
        carta1 = self.jugador1.jugar_carta()
        carta2 = self.jugador2.jugar_carta()
        print(f"{self.jugador1.nombre} jugó {carta1}")
        print(f"{self.jugador2.nombre} jugó {carta2}")
        
        if carta1.valor > carta2.valor:
            ganador = self.jugador1
        elif carta2.valor > carta1.valor:
            ganador = self.jugador2
        else:
            ganador = None
        
        if ganador:
            print(f"{ganador.nombre} gana la ronda")
            ganador.sumar_punto()
        else:
            print("Empate en esta ronda")
        
        return ganador

    def determinar_ganador(self):
        if self.puntaje[self.jugador1] > self.puntaje[self.jugador2]:
            return self.jugador1
        elif self.puntaje[self.jugador2] > self.puntaje[self.jugador1]:
            return self.jugador2
        else:
            return None

    def mostrar_puntaje(self, mensaje):
        print(f"\n{mensaje}")
        print(f"{self.jugador1.nombre}: {self.puntaje[self.jugador1]} puntos")
        print(f"{self.jugador2.nombre}: {self.puntaje[self.jugador2]} puntos")

    def jugar(self):
        for ronda in range(1, 7):
            print(f"\n--- Ronda {ronda} ---")
            self.mazo.mezclar()
            self.jugador1.recibir_cartas(self.mazo.repartir(3))
            self.jugador2.recibir_cartas(self.mazo.repartir(3))
            
            for _ in range(3):
                ganador_ronda = self.jugar_ronda()
                if ganador_ronda:
                    self.puntaje[ganador_ronda] += 1
            
            self.mostrar_puntaje("Puntaje actual:")
            
            if self.puntaje[self.jugador1] >= 15 or self.puntaje[self.jugador2] >= 15:
                break
        
        ganador = self.determinar_ganador()
        
        if ganador:
            print(f"\n¡{ganador.nombre} ha ganado la partida!")
        else:
            print("\n¡La partida ha terminado en empate!")
        
        self.mostrar_puntaje("Puntaje final:")

jugador1 = Jugador("Juan")
jugador2 = Jugador("Pedro")
p = Partida(jugador1, jugador2)
p.jugar()


--- Ronda 1 ---
Juan jugó 12 de basto
Pedro jugó 10 de espada
Juan gana la ronda
Juan jugó 2 de oro
Pedro jugó 5 de basto
Juan gana la ronda
Juan jugó 5 de espada
Pedro jugó 10 de copa
Pedro gana la ronda

Puntaje actual:
Juan: 2 puntos
Pedro: 1 puntos

--- Ronda 2 ---
Juan jugó 1 de espada
Pedro jugó 4 de copa
Juan gana la ronda
Juan jugó 7 de copa
Pedro jugó 3 de basto
Pedro gana la ronda
Juan jugó 12 de oro
Pedro jugó 7 de oro
Pedro gana la ronda

Puntaje actual:
Juan: 3 puntos
Pedro: 3 puntos

--- Ronda 3 ---
Juan jugó 12 de copa
Pedro jugó 11 de oro
Juan gana la ronda
Juan jugó 10 de basto
Pedro jugó 11 de copa
Pedro gana la ronda
Juan jugó 5 de copa
Pedro jugó 6 de espada
Pedro gana la ronda

Puntaje actual:
Juan: 4 puntos
Pedro: 5 puntos

--- Ronda 4 ---
Juan jugó 3 de oro
Pedro jugó 1 de oro
Juan gana la ronda
Juan jugó 6 de oro
Pedro jugó 11 de basto
Pedro gana la ronda
Juan jugó 3 de copa
Pedro jugó 10 de oro
Juan gana la ronda

Puntaje actual:
Juan: 6 puntos
Pedro: 6 puntos